In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/TimesNet/
!ls

Mounted at /content/drive
/content/drive/MyDrive/TimesNet
ohlcv_BINANCE_ETHUSDT.P.csv  TimesNet.ipynb


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
class InceptionBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.1):
        super().__init__()
        self.branch1 = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 7), padding=(0, 3))  # Extended time horizon for short-term patterns
        self.branch2 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 5), padding=(1, 2))  # Balanced time-feature relationships
        self.branch3 = nn.Conv2d(in_channels, out_channels, kernel_size=(7, 3), padding=(3, 1))  # Enhanced cross-feature interactions
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(out_channels * 3)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out = torch.cat([out1, out2, out3], dim=1)
        out = self.relu(self.bn(out))
        return self.dropout(out)

In [ ]:
class TimesBlock(nn.Module):
    def __init__(self, d_model, top_k=3, dropout=0.1):
        super().__init__()
        self.top_k = top_k
        self.d_model = d_model
        self.inception = InceptionBlock2D(1, 1, dropout)
        self.output_proj = nn.Linear(3, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, C, T = x.shape
        freq = torch.fft.rfft(x, dim=-1)
        amplitude = freq.abs()[:, :, 1:]
        topk_indices = torch.topk(amplitude, self.top_k, dim=-1)[1]
        periods = (T // (topk_indices + 1)).clamp(min=2)
        output = torch.zeros_like(x)

        for k in range(self.top_k):
            p = periods[:, :, k]
            median_p = int(torch.median(p).item())
            seq_len = (T // median_p) * median_p
            views = x[:, :, :seq_len].reshape(B, C, -1, median_p)
            views = views.view(B * C, 1, views.shape[2], median_p)
            conv_out = self.inception(views)
            conv_out = conv_out.mean(dim=[2, 3])
            conv_out = conv_out.view(B, C, -1)
            proj_out = self.output_proj(conv_out)
            output += proj_out

        return self.dropout(output / self.top_k) + x

In [ ]:
class TimesNet(nn.Module):
    def __init__(self, d_model=64, num_blocks=3, dropout=0.1):
        super().__init__()
        self.blocks = nn.ModuleList([TimesBlock(d_model, dropout=dropout) for _ in range(num_blocks)])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

In [ ]:
class TimesNetForecaster(nn.Module):
    def __init__(self, input_channels, seq_len, pred_len, d_model=128, num_blocks=4, dropout=0.2):
        super().__init__()
        self.feature_embedding = nn.Sequential(
            nn.Linear(input_channels, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.input_proj = nn.Sequential(
            nn.Linear(seq_len, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.backbone = TimesNet(d_model=d_model, num_blocks=num_blocks, dropout=dropout)
        self.head = nn.Sequential(
            nn.Linear(d_model, d_model * 2),
            nn.LayerNorm(d_model * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 2, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, pred_len)
        )

    def forward(self, x):
        x = self.feature_embedding(x)   # [B, T, d_model]
        x = x.transpose(1, 2)           # [B, d_model, T]
        x = self.input_proj(x)          # [B, d_model, d_model]
        x = self.backbone(x)
        x = x.transpose(1, 2)           # [B, d_model, d_model]
        return self.head(x)

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len, pred_len, target_column=-1, stride=30):
        self.input_data = torch.tensor(data[:, :-1], dtype=torch.float32)
        self.target_data = torch.tensor(data[:, target_column], dtype=torch.float32)
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.indices = list(range(0, len(data) - seq_len - pred_len + 1, stride))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        x = self.input_data[i:i+self.seq_len]
        y = self.target_data[i+self.seq_len:i+self.seq_len+self.pred_len]
        return x, y

In [ ]:
def train_model(model, train_data, val_data, seq_len, pred_len, batch_size=64, epochs=5, lr=1e-3, stride=30, target_col=-1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_loader = DataLoader(TimeSeriesDataset(train_data, seq_len, pred_len, target_col, stride), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TimeSeriesDataset(val_data, seq_len, pred_len, target_col, stride), batch_size=batch_size)


    for epoch in range(epochs):
        model.train()
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            pred = model(X).mean(dim=1)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_postfix(loss=loss.item())

        trues, preds = [], []
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                X, y = X.to(device), y.to(device)
                pred = model(X).mean(dim=1)
                val_loss += criterion(pred, y).item()
                preds.append(pred.cpu().numpy())
                trues.append(y.cpu().numpy())

        preds = np.concatenate(preds)
        trues = np.concatenate(trues)
        r2 = r2_score(trues, preds)
        print(f"Epoch {epoch+1}, Val Loss: {val_loss / len(val_loader):.4f}, R2: {r2:.4f}")

    return model

In [ ]:
def evaluate_model(model, test_data, seq_len, pred_len, batch_size=128, stride=30, target_col=-1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    dataset = TimeSeriesDataset(test_data, seq_len, pred_len, target_col, stride)
    loader = DataLoader(dataset, batch_size=batch_size)

    preds, trues = [], []
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            out = model(X).mean(dim=1)
            preds.append(out.cpu().numpy())
            trues.append(y.cpu().numpy())

    preds = np.concatenate(preds)
    trues = np.concatenate(trues)

    mse = mean_squared_error(trues, preds)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    print(f"Test MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

In [ ]:
df = pd.read_csv("ohlcv_BINANCE_ETHUSDT.P.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['minute']     = df['timestamp'].dt.minute
df['hour']       = df['timestamp'].dt.hour
df['day']        = df['timestamp'].dt.day
df['dayofweek']  = df['timestamp'].dt.dayofweek
df['month']      = df['timestamp'].dt.month

df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)

df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

In [ ]:
print(df.columns)

Index(['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'minute', 'hour', 'day', 'dayofweek', 'month', 'hour_sin', 'hour_cos',
       'minute_sin', 'minute_cos', 'dayofweek_sin', 'dayofweek_cos'],
      dtype='object')


In [ ]:
if __name__ == "__main__":
    print("Loading data...")
    features = [
        'open', 'high', 'low', 'close', 'volume',
       'minute', 'hour', 'day', 'dayofweek', 'month', 'hour_sin', 'hour_cos',
       'minute_sin', 'minute_cos', 'dayofweek_sin', 'dayofweek_cos'
    ]

    input_data = df[features].values
    scaler = StandardScaler()
    input_data = scaler.fit_transform(input_data)

    df['pre_return'] = (1000 * df['close'].pct_change()).fillna(0)
    pre_return_col = df['pre_return'].values.reshape(-1, 1)
    input_data = np.hstack((input_data, pre_return_col))
    target_data = df['pre_return']


    data = np.column_stack([input_data, target_data])

    train_size = int(len(data) * 0.7)
    val_size = int(len(data) * 0.15)
    train_data = data[:train_size]
    val_data = data[train_size:train_size+val_size]
    test_data = data[train_size+val_size:]

    print("TimesNetForecaster model")

    model = TimesNetForecaster(
        input_channels=len(features) + 1,
        seq_len=360,
        pred_len=60,
        d_model=128,      # increased from 64
        num_blocks=4,     # increased from 3
        dropout=0.2       # increased from 0.1
    )

    print("Training model...")
    model = train_model(
        model,
        train_data,
        val_data,
        seq_len=360,
        pred_len=60,
        batch_size=32,    # reduced from 64
        epochs=5,        # increased from 5
        lr=1e-4           # decreased from 1e-3
    )

    print("Evaluating model...")
    evaluate_model(model, test_data, seq_len=360, pred_len=60)


Loading data...
TimesNetForecaster model
Training model...


Epoch 1/5 [Train]:   4%|▍         | 21/473 [00:02<00:52,  8.65it/s, loss=1.62]


KeyboardInterrupt: 